# NF26/AI07 - TD2 : Création d'un Modèle en Étoile



In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [3]:
spark = SparkSession.builder.getOrCreate()

## Introduction

L'objectif de ce TD est d'implémenter un modèle Étoile à partir de 6 différentes bases de données transactionnelles non-normailisées pour procéder des requêtes visant à avoir une aperçu sur les données d'un Hôpital. Les 6 diffrérentes bases de données transactionnelles données sont :
- `data_administration`: Ensemble des données administratives liées à chaque consultation.
- `data_medecins`: Ensemble des données liées aux informations associées à chaque Médecin.
- `data_diagnostics`: Ensemble des données liées aux diagnostic de chaque patient lors de chaque consultation.
- `data_treatments`: Ensemble des données liées aux descriptions de chaque traitement.
- `data_medicaments`: Ensemble des informations caractérisant chaque Médicament.
- `data_chambres`: Ensemble des informations caractérisant chaque Chambre de l'Hôpital.

On souhaite notamment pouvoir répondre aux requêtes suivantes :
- "*Quel a été l'âge moyen des patients qui ont eu une certaine Pathology durant une certaine période ?*".
- "*Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine Pathology durant une certaine période ?*".
- "*Combien de chambres ont accueilli des patients diagnostiqués d'une certaine Pathology durant une certaine période ?*".
- "*Quelle est la proportion de medecins (par spécialité) qui ont dignostiqué une certaine `Pathology` durant une certaine période ?*".


## Lecture des données

Les cellules ci-dessous permettent de lire les différentes bases de données transactionnelles en format `pyspark.pandas.frame.DataFrame`.


In [6]:
psdf_administration = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD2/data_administration.csv', index_col='KeyConsult')
psdf_administration.head(10)

,KeyPatient,Name,FirstName,NumSecu,Date_In,Pathology,KeyChambre,ChambreNumber,Date_Out
KeyConsult,,,,,,,,,
KeyConsult45056,KeyPatient10546,Name10546,FistName10546,NS45057,2023-11-24 23:00:00,Pathology22,KeyChambre81,C081,2023-11-25 23:00:00
KeyConsult45057,KeyPatient45482,Name45482,FistName45482,NS45058,2023-11-24 23:00:00,Pathology34,KeyChambre102,C102,2023-11-25 23:00:00
KeyConsult45058,KeyPatient37050,Name37050,FistName37050,NS45059,2023-11-24 23:00:00,Pathology84,KeyChambre22,C022,2023-12-04 23:00:00
KeyConsult45059,KeyPatient39793,Name39793,FistName39793,NS45060,2023-11-24 23:00:00,Pathology78,KeyChambre4,C004,2023-11-30 23:00:00
KeyConsult45060,KeyPatient17652,Name17652,FistName17652,NS45061,2023-11-24 23:00:00,Pathology15,KeyChambre29,C029,2023-11-25 23:00:00
KeyConsult45061,KeyPatient40034,Name40034,FistName40034,NS45062,2023-11-24 23:00:00,Pathology7,KeyChambre60,C060,2023-11-25 23:00:00
KeyConsult45062,KeyPatient38237,Name38237,FistName38237,NS45063,2023-11-24 23:00:00,Pathology7,KeyChambre5,C005,2023-11-26 23:00:00
KeyConsult45063,KeyPatient17519,Name17519,FistName17519,NS45064,2023-11-24 23:00:00,Pathology67,KeyChambre84,C084,2023-12-02 23:00:00
KeyConsult45064,KeyPatient28813,Name28813,FistName28813,NS45065,2023-11-24 23:00:00,Pathology56,KeyChambre51,C051,2023-11-25 23:00:00


In [80]:
psdf_medecins = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD2/data_medecins.csv', index_col='KeyMedecin')
psdf_medecins.head(10)

,NameMedecin,FirstNameMedecin,SpecialityMedecin
KeyMedecin,,,
keyMedecin26,NameMedecin26,FirstNameMedecin26,Generaliste
keyMedecin27,NameMedecin27,FirstNameMedecin27,Chirurgien
keyMedecin28,NameMedecin28,FirstNameMedecin28,Dermatologiste
keyMedecin29,NameMedecin29,FirstNameMedecin29,Generaliste
keyMedecin16,NameMedecin16,FirstNameMedecin16,Chirurgien
keyMedecin17,NameMedecin17,FirstNameMedecin17,Chirurgien
keyMedecin18,NameMedecin18,FirstNameMedecin18,Chirurgien
keyMedecin19,NameMedecin19,FirstNameMedecin19,Generaliste
keyMedecin6,NameMedecin6,FirstNameMedecin6,Dermatologiste


In [8]:
psdf_diagnostics = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD2/data_diagnostics.csv', index_col='KeyConsult')
psdf_diagnostics.head(10)

,KeyMedecin,KeyPatient,NamePatient,FirstNamePatient,NumSecu,Age,Weight,Temperature,Tension,Diabete,Pathology,KeyTreatment
KeyConsult,,,,,,,,,,,,
KeyConsult45056,keyMedecin26,KeyPatient10546,Name10546,FistName10546,NS45057,24,58.759522,37.459899,135.700753,0,Pathology22,KeyTreatment45056
KeyConsult45057,keyMedecin22,KeyPatient45482,Name45482,FistName45482,NS45058,19,58.422513,37.488268,95.790925,0,Pathology34,KeyTreatment45057
KeyConsult45058,keyMedecin14,KeyPatient37050,Name37050,FistName37050,NS45059,50,80.125748,38.023957,89.173819,1,Pathology84,KeyTreatment45058
KeyConsult45059,keyMedecin29,KeyPatient39793,Name39793,FistName39793,NS45060,76,69.306556,39.966692,116.606928,0,Pathology78,KeyTreatment45059
KeyConsult45060,keyMedecin18,KeyPatient17652,Name17652,FistName17652,NS45061,22,56.173338,37.463554,161.955995,0,Pathology15,KeyTreatment45060
KeyConsult45061,keyMedecin21,KeyPatient40034,Name40034,FistName40034,NS45062,16,55.421938,37.554036,153.418985,0,Pathology7,KeyTreatment45061
KeyConsult45062,keyMedecin18,KeyPatient38237,Name38237,FistName38237,NS45063,22,63.067835,37.213361,171.359467,0,Pathology7,KeyTreatment45062
KeyConsult45063,keyMedecin2,KeyPatient17519,Name17519,FistName17519,NS45064,34,58.652805,38.459865,107.910361,0,Pathology67,KeyTreatment45063
KeyConsult45064,keyMedecin21,KeyPatient28813,Name28813,FistName28813,NS45065,27,59.233520,37.535722,123.634315,0,Pathology56,KeyTreatment45064


In [9]:
psdf_treatments = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD2/data_treatments.csv', index_col='KeyTreatment')
psdf_treatments.head(10)

,KeyMedicament,QuantityMedicament
KeyTreatment,,
KeyTreatment20495,KeyMed513,1
KeyTreatment20495,KeyMed852,2
KeyTreatment20495,KeyMed265,1
KeyTreatment20496,KeyMed82,2
KeyTreatment20497,KeyMed67,1
KeyTreatment20497,KeyMed21,3
KeyTreatment20498,KeyMed697,1
KeyTreatment20499,KeyMed2,2
KeyTreatment20500,KeyMed47,2


In [10]:
psdf_medicaments = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD2/data_medicaments.csv', index_col='KeyMedicament')
psdf_medicaments.head(10)

,NameMedicament
KeyMedicament,
KeyMed913,Medicament913
KeyMed914,Medicament914
KeyMed915,Medicament915
KeyMed916,Medicament916
KeyMed917,Medicament917
KeyMed918,Medicament918
KeyMed919,Medicament919
KeyMed920,Medicament920
KeyMed921,Medicament921


In [11]:
psdf_chambres = ps.read_csv('/content/drive/MyDrive/Colab Notebooks/TD2/data_chambres.csv', index_col='KeyChambre')
psdf_chambres.head(10)

,ChambreNumber
KeyChambre,
KeyChambre89,C089
KeyChambre90,C090
KeyChambre91,C091
KeyChambre92,C092
KeyChambre93,C093
KeyChambre94,C094
KeyChambre95,C095
KeyChambre96,C096
KeyChambre97,C097


Afin de pouvoir réaliser notre objectif, nous pourrons convertir ces données en format `pyspark.sql.dataframe.DataFrame`.

In [12]:
sdf_administration = psdf_administration.to_spark(index_col='KeyConsult')
sdf_medecins = psdf_medecins.to_spark(index_col='KeyMedecin')
sdf_diagnostics = psdf_diagnostics.to_spark(index_col='KeyConsult')
sdf_treatments = psdf_treatments.to_spark(index_col='KeyTreatment')
sdf_medicaments = psdf_medicaments.to_spark(index_col='KeyMedicament')
sdf_chambres = psdf_chambres.to_spark(index_col='KeyChambre')

In [13]:
from pyspark.sql.functions import *

## Exercices

On souhaite créer un modèle en étoile permettant de procéder facilement les requêtes suivantes :
- "*Quel a été l'âge moyen des patients qui ont eu une certaine Pathology durant une certaine période ?*".
- "*Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine Pathology durant une certaine période ?*".
- "*Combien de chambres ont des patients diagnostiqués d'une certaine Pathology durant une certaine période ?*".
- "*Quelle est la proportion de medecins (par spécialité) qui ont dignostiqué une certaine `Pathology` durant une certaine période ?*".

Pour essayer de répondre à toutes ces questions, nous allons considérer le modèle étoile suivant. Pour cela, nous pourrons par exemple utiliser l'aide disponible ici : https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html

![title](Schemas_Etoile_TD_new.pdf)

**Question 1.** À partir des 6 bases de données transactionnelles, créer la `table de faits` associée aux `Consultations`.

In [23]:
t1 = sdf_administration.join(sdf_diagnostics, "KeyConsult")
facts = t1.select("*").withColumn("KeyDates", monotonically_increasing_id())
facts = facts.select("KeyConsult", sdf_administration.KeyPatient, "KeyMedecin", "KeyTreatment", "KeyDates", "KeyChambre")
facts.show()

+---------------+---------------+------------+-----------------+--------+-------------+
|     KeyConsult|     KeyPatient|  KeyMedecin|     KeyTreatment|KeyDates|   KeyChambre|
+---------------+---------------+------------+-----------------+--------+-------------+
|KeyConsult45056|KeyPatient10546|keyMedecin26|KeyTreatment45056|       0| KeyChambre81|
|KeyConsult45057|KeyPatient45482|keyMedecin22|KeyTreatment45057|       1|KeyChambre102|
|KeyConsult45058|KeyPatient37050|keyMedecin14|KeyTreatment45058|       2| KeyChambre22|
|KeyConsult45059|KeyPatient39793|keyMedecin29|KeyTreatment45059|       3|  KeyChambre4|
|KeyConsult45060|KeyPatient17652|keyMedecin18|KeyTreatment45060|       4| KeyChambre29|
|KeyConsult45061|KeyPatient40034|keyMedecin21|KeyTreatment45061|       5| KeyChambre60|
|KeyConsult45062|KeyPatient38237|keyMedecin18|KeyTreatment45062|       6|  KeyChambre5|
|KeyConsult45063|KeyPatient17519| keyMedecin2|KeyTreatment45063|       7| KeyChambre84|
|KeyConsult45064|KeyPatient28813

**Question 2.** À partir des 6 bases de données transactionnelles, créer la `table de dimension` associée aux `Patients`.

In [24]:
dim_patients = sdf_diagnostics.select("KeyPatient", "KeyConsult", "NamePatient", "FirstNamePatient", "NumSecu", "Age", "Weight", "Temperature", "Tension", "Diabete", "Pathology")
dim_patients.show()

+---------------+---------------+-----------+----------------+-------+---+------------------+------------------+------------------+-------+-----------+
|     KeyPatient|     KeyConsult|NamePatient|FirstNamePatient|NumSecu|Age|            Weight|       Temperature|           Tension|Diabete|  Pathology|
+---------------+---------------+-----------+----------------+-------+---+------------------+------------------+------------------+-------+-----------+
|KeyPatient10546|KeyConsult45056|  Name10546|   FistName10546|NS45057| 24| 58.75952185728825| 37.45989878643732| 135.7007525481543|      0|Pathology22|
|KeyPatient45482|KeyConsult45057|  Name45482|   FistName45482|NS45058| 19| 58.42251330648344| 37.48826750167022| 95.79092452672673|      0|Pathology34|
|KeyPatient37050|KeyConsult45058|  Name37050|   FistName37050|NS45059| 50|  80.1257478193804| 38.02395668303052| 89.17381944535508|      1|Pathology84|
|KeyPatient39793|KeyConsult45059|  Name39793|   FistName39793|NS45060| 76|  69.306556451

**Question 3.** À partir des 6 bases de données transactionnelles, créer la `table de dimension` associée aux `Dates`.

In [21]:
dates = sdf_administration.select("Date_In", "Date_Out")
dim_dates = dates.select("*").withColumn("KeyDates", monotonically_increasing_id()).select("KeyDates", "Date_In", "Date_Out")
dim_dates.show()

+--------+-------------------+-------------------+
|KeyDates|            Date_In|           Date_Out|
+--------+-------------------+-------------------+
|       0|2023-11-24 23:00:00|2023-11-25 23:00:00|
|       1|2023-11-24 23:00:00|2023-11-25 23:00:00|
|       2|2023-11-24 23:00:00|2023-12-04 23:00:00|
|       3|2023-11-24 23:00:00|2023-11-30 23:00:00|
|       4|2023-11-24 23:00:00|2023-11-25 23:00:00|
|       5|2023-11-24 23:00:00|2023-11-25 23:00:00|
|       6|2023-11-24 23:00:00|2023-11-26 23:00:00|
|       7|2023-11-24 23:00:00|2023-12-02 23:00:00|
|       8|2023-11-24 23:00:00|2023-11-25 23:00:00|
|       9|2023-11-24 23:00:00|2023-11-25 23:00:00|
|      10|2023-11-24 23:00:00|2023-11-25 23:00:00|
|      11|2023-11-24 23:00:00|2023-11-25 23:00:00|
|      12|2023-11-24 23:00:00|2023-11-29 23:00:00|
|      13|2023-11-24 23:00:00|2023-12-01 23:00:00|
|      14|2023-11-24 23:00:00|2023-12-03 23:00:00|
|      15|2023-11-24 23:00:00|2023-11-25 23:00:00|
|      16|2023-11-24 23:00:00|2

**Question 4.** À partir des 6 bases de données transactionnelles, créer la `table de dimension` associée aux `Medecins`.

In [25]:
dim_medecin = sdf_medecins.select("*")
dim_medecin.show()

+------------+-------------+------------------+-----------------+
|  KeyMedecin|  NameMedecin|  FirstNameMedecin|SpecialityMedecin|
+------------+-------------+------------------+-----------------+
|keyMedecin26|NameMedecin26|FirstNameMedecin26|      Generaliste|
|keyMedecin27|NameMedecin27|FirstNameMedecin27|       Chirurgien|
|keyMedecin28|NameMedecin28|FirstNameMedecin28|   Dermatologiste|
|keyMedecin29|NameMedecin29|FirstNameMedecin29|      Generaliste|
|keyMedecin16|NameMedecin16|FirstNameMedecin16|       Chirurgien|
|keyMedecin17|NameMedecin17|FirstNameMedecin17|       Chirurgien|
|keyMedecin18|NameMedecin18|FirstNameMedecin18|       Chirurgien|
|keyMedecin19|NameMedecin19|FirstNameMedecin19|      Generaliste|
| keyMedecin6| NameMedecin6| FirstNameMedecin6|   Dermatologiste|
| keyMedecin7| NameMedecin7| FirstNameMedecin7|       Chirurgien|
| keyMedecin8| NameMedecin8| FirstNameMedecin8|      Generaliste|
| keyMedecin9| NameMedecin9| FirstNameMedecin9|      Generaliste|
|keyMedeci

**Question 5.** À partir des 6 bases de données transactionnelles, créer la `table de dimension` associée aux `Traitements`.

In [27]:
dim_traitement = sdf_treatments.join(sdf_medicaments, "KeyMedicament").select("KeyTreatment", "KeyMedicament", "QuantityMedicament", "NameMedicament")
dim_traitement.show()

+-----------------+-------------+------------------+--------------+
|     KeyTreatment|KeyMedicament|QuantityMedicament|NameMedicament|
+-----------------+-------------+------------------+--------------+
|KeyTreatment20495|    KeyMed513|                 1| Medicament513|
|KeyTreatment20495|    KeyMed852|                 2| Medicament852|
|KeyTreatment20495|    KeyMed265|                 1| Medicament265|
|KeyTreatment20496|     KeyMed82|                 2|  Medicament82|
|KeyTreatment20497|     KeyMed67|                 1|  Medicament67|
|KeyTreatment20497|     KeyMed21|                 3|  Medicament21|
|KeyTreatment20498|    KeyMed697|                 1| Medicament697|
|KeyTreatment20499|      KeyMed2|                 2|   Medicament2|
|KeyTreatment20500|     KeyMed47|                 2|  Medicament47|
|KeyTreatment20500|    KeyMed581|                 3| Medicament581|
|KeyTreatment20501|      KeyMed3|                 3|   Medicament3|
|KeyTreatment20502|    KeyMed800|               

**Question 6.** À partir des 6 bases de données transactionnelles, créer la `table de dimension` associée aux `Chambres`.

In [28]:
dim_chambre = sdf_chambres.select("*")
dim_chambre.show()

+-------------+-------------+
|   KeyChambre|ChambreNumber|
+-------------+-------------+
| KeyChambre89|         C089|
| KeyChambre90|         C090|
| KeyChambre91|         C091|
| KeyChambre92|         C092|
| KeyChambre93|         C093|
| KeyChambre94|         C094|
| KeyChambre95|         C095|
| KeyChambre96|         C096|
| KeyChambre97|         C097|
| KeyChambre98|         C098|
| KeyChambre99|         C099|
|KeyChambre100|         C100|
|KeyChambre101|         C101|
|KeyChambre102|         C102|
|KeyChambre103|         C103|
|KeyChambre104|         C104|
| KeyChambre33|         C033|
| KeyChambre34|         C034|
| KeyChambre35|         C035|
| KeyChambre36|         C036|
+-------------+-------------+
only showing top 20 rows



**Question 7.** Écrire un code qui permette de calculer l'âge moyen des patients qui ont eu une certaine `Pathology` durant un certain mois. Nous pourrons par exemple considérer la `Pathology75` puis la `Pathology15` durant le mois de mars 2023.

In [41]:
t1 = dim_patients.filter(dim_patients.Pathology == "Pathology75")
t1 = t1.join(facts, "KeyPatient").join(dim_dates, "KeyDates").select("Age", "Date_In")
t1 = t1.filter((month(col("Date_In")) == 3) & (year(col("Date_In")) == 2023))
t1.select(mean(t1.Age)).show()

t2 = dim_patients.filter(dim_patients.Pathology == "Pathology15")
t2 = t2.join(facts, "KeyPatient").join(dim_dates, "KeyDates").select("Age", "Date_In")
t2 = t2.filter((month(col("Date_In")) == 3) & (year(col("Date_In")) == 2023))
t2.select(mean(t2.Age)).show()

+--------+
|avg(Age)|
+--------+
|  70.625|
+--------+

+------------------+
|          avg(Age)|
+------------------+
|20.247191011235955|
+------------------+



**Question 8.** Quel Médicament a été le plus prescrit (en terme de quantité) pour une certaine `Pathology` durant une certaine période ? Nous pourrons par exemple considérer la `Pathology43` durant le mois de février 2023.

In [53]:
t3 = dim_traitement.join(facts, "KeyTreatment").join(dim_patients, "KeyPatient").join(dim_dates, "KeyDates")
t3 = t3.filter(t3.Pathology == "Pathology43").filter((month(col("Date_In")) == 2) & (year(col("Date_In")) == 2023)).select("KeyMedicament", "QuantityMedicament")
t3_total = t3.groupBy("KeyMedicament").agg(sum("QuantityMedicament").alias("TotalQuantity")).orderBy("TotalQuantity", ascending=False)
t3_total.show()

+-------------+-------------+
|KeyMedicament|TotalQuantity|
+-------------+-------------+
|    KeyMed362|            6|
|    KeyMed518|            6|
|    KeyMed135|            5|
|     KeyMed10|            5|
|    KeyMed524|            5|
|    KeyMed498|            4|
|     KeyMed21|            4|
|    KeyMed363|            3|
|    KeyMed380|            3|
|    KeyMed744|            3|
|    KeyMed323|            3|
|    KeyMed982|            3|
|    KeyMed138|            3|
|    KeyMed796|            3|
|    KeyMed266|            3|
|    KeyMed346|            3|
|    KeyMed990|            3|
|    KeyMed641|            3|
|     KeyMed20|            3|
|    KeyMed753|            3|
+-------------+-------------+
only showing top 20 rows



**Question 9.** Combien de `chambres` ont accueilli des patients diagnostiqués d'une certaine `Pathology` durant une certaine période ? Nous pourrons par exemple considérer la `Pathology54` durant le mois de novembre 2023.

In [85]:
pathology = dim_patients.join(facts, "KeyPatient").join(dim_dates, "KeyDates").filter(col("Pathology") == "Pathology54").filter((month(col("Date_In")) == 11) & (year(col("Date_In")) == 2023))
pathology_all = pathology.groupBy("Pathology").agg(count("KeyChambre"))
pathology_all.show()

pathology_dist = pathology.groupBy("Pathology").agg(countDistinct("KeyChambre"))
pathology_dist.show()

+-----------+-----------------+
|  Pathology|count(KeyChambre)|
+-----------+-----------------+
|Pathology54|               88|
+-----------+-----------------+

+-----------+--------------------------+
|  Pathology|count(DISTINCT KeyChambre)|
+-----------+--------------------------+
|Pathology54|                        50|
+-----------+--------------------------+



**Question Bonus.** Déterminer puis afficher sur une figure la proportion de medecins (par spécialité) qui ont dignostiqué une certaine `Pathology` durant une certaine période.

In [84]:
t = dim_medecin.join(facts, "KeyMedecin").join(dim_patients, "KeyPatient").join(dim_dates, "KeyDates")
filtered = t.filter(col("Pathology") == "Pathology75").filter((month(col("Date_In")) == 11) & (year(col("Date_In")) == 2023))
filtered = filtered.groupBy("SpecialityMedecin").agg(count("KeyMedecin").alias("count_filtered"))

all = t.filter((month(col("Date_In")) == 11) & (year(col("Date_In")) == 2023))
all = all.groupBy("SpecialityMedecin").agg(count("KeyMedecin").alias("count_all"))

main = filtered.join(all, "SpecialityMedecin")
main = main.withColumn("Proportion", col("count_filtered") / col("count_all"))

main.show()




+-----------------+--------------+---------+--------------------+
|SpecialityMedecin|count_filtered|count_all|          Proportion|
+-----------------+--------------+---------+--------------------+
|   Dermatologiste|            19|     1378|0.013788098693759071|
|         Dentiste|             2|      248|0.008064516129032258|
|       Chirurgien|            34|     2995|0.011352253756260434|
|      Generaliste|            34|     3660|0.009289617486338797|
+-----------------+--------------+---------+--------------------+

